In [1]:
import ase.io
from LDHPbuilder.perovskite_builder import OrganicMolecule, InorganicMonolayer, PerovskiteBuilder
import ase.units as units
import numpy as np
np.random.seed(0)

This Notebooks shows how to use LDHP builder to make a set of candidate structures for a 2D hybrid perovskite, and then relax them with a MACE potential.

# 1) Making Candidate Structures

### 1.1) Starting Information

Suppose you want to predict the structure formed when a given organic cation is combined with chosen inorganic framework. Currently, our code assumes that the inorganic frameowrk will be a monolayer of corner sharing octahedra of the form $BX_4$. To get started, you therefore need:
- a cation geometry in an `ase.Atoms` object, with a known charge
- a choice of $B$-site
- a choice of $X$-site
- an indended unit cell size to search over

In [2]:
# load the geometry of an organic cation, for instance from an xyz file:
cation_atoms = ase.io.read('cation_plus_2.xyz')

### 1.2) Construct an `InorganicMonolayer` object and an `OrganicMolecule` object

An `InorganicMonolayer` is an atoms object of a monolayer in a wrapper which contains some useful information for structure generation. You can specify the size of the unit cell in the in-plane directions, through the parameter `num_unit_cell_octahedra`.

Similarly, an `OrganicMolecule` object wraps the geometry of the molecule with some useful information and functions. The charge must be specified manually.

In [3]:
# we will make  monolayer of PbI_4 octahedra, with 2 lead atoms per unit cell
monolayer = InorganicMonolayer.from_species_specification('Pb', 'I', num_unit_cell_octahedra=2)

molecule = OrganicMolecule(
    cation_atoms,
    2 # charge - this cation is doubly charged
)

### 1.3) Generate Samples

In [4]:
pb = PerovskiteBuilder()

samples = pb.generate_homogeneous_perovskite_samples( # homogeneous since just one kind of molecule
    monolayer, 
    molecule, 
    num_layers=1, # should be a power of 2, but 4 already implies a very large search task
    num_samples=10, 
    max_num_attempts=15, 
    stacking_method='total_thickness' # this can be either 'total_thickness' or 'half_thickness'. use half_thickness for long, thin +1 molecules
)

# save the samples to a file to inspect them
# ase.io.write('samples.xyz', samples)

found 1 structures after 0 attempts
found 2 structures after 1 attempts
found 3 structures after 2 attempts
found 4 structures after 3 attempts
found 5 structures after 4 attempts
found 6 structures after 5 attempts
found 7 structures after 6 attempts
found 8 structures after 7 attempts
found 9 structures after 8 attempts
found 10 structures after 9 attempts
generated 10 samples after 10 attempts


# 2) Relaxing Structures with the MACE calculator

In [5]:
from mace.calculators.mace import MACECalculator
from ase.constraints import UnitCellFilter
from ase.optimize.precon import Exp, PreconLBFGS
from copy import deepcopy

/Users/will/miniconda3/envs/nomace/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Relax the Structures

We just relax one structure as a demonstration.

NOTE: 
1. The MACE model for 2D perovksites, which was trained for this purpose, can be found via the reference in the README.
2. MACE is designed to run on GPU, so relaxations will be mcuh slower on CPU

In [6]:
# copy because relaxations are done in place
ats = deepcopy(samples[0])

import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# declare mace calculator - see MACE documentation for details
ats.calc = MACECalculator(
    model_path='path/to/mace_model.model',
    default_dtype='float64',
    device=device,
)

# choose your favourite optimizer 
# a good strategy is to first relax with an external pressure to prevent vacuum regions
ucf = UnitCellFilter(ats, mask=[True for i in range(6)], scalar_pressure=1.0 * units.GPa)
dyn = PreconLBFGS(
    ucf, 
    precon='Exp', 
    trajectory='relaxation_1.traj',
)

# run the dynamics
force_thr = 1e-1  # force tolerance in eV/A
dyn.run(fmax=force_thr)

# now relax without pressure
ucf = UnitCellFilter(ats, mask=[True for i in range(6)], scalar_pressure=0.0 * units.GPa)
dyn = PreconLBFGS(
    ucf, 
    precon='Exp', 
    trajectory='relaxation_2.traj'
)
force_thr = 1e-2
dyn.run(fmax=force_thr)

ats.info['final_energy'] = ats.get_potential_energy()

PreconLBFGS:   0  12:09:32     -331.047680       7.5923       0.0629
PreconLBFGS:   1  12:09:34     -332.109973       7.0576       0.0591
PreconLBFGS:   2  12:09:34     -333.017758       6.7628       0.0557
PreconLBFGS:   3  12:09:34     -333.960401       6.2168       0.0523
PreconLBFGS:   4  12:09:35     -334.811829       6.1174       0.0488
PreconLBFGS:   5  12:09:35     -335.818053       5.3546       0.0452
PreconLBFGS:   6  12:09:36     -336.710886       5.1013       0.0413
PreconLBFGS:   7  12:09:36     -337.736173       4.5544       0.0370
PreconLBFGS:   8  12:09:36     -338.775595       4.0692       0.0322
PreconLBFGS:   9  12:09:37     -339.739506       3.6928       0.0276
PreconLBFGS:  10  12:09:37     -340.347963       3.2282       0.0244
PreconLBFGS:  11  12:09:38     -340.781709       3.0467       0.0222
PreconLBFGS:  12  12:09:38     -341.106239       2.7891       0.0204
PreconLBFGS:  13  12:09:39     -341.363468       2.6607       0.0190
PreconLBFGS:  14  12:09:39     -34

In [10]:
full_traj = ase.io.read('relaxation_1.traj', ':') + ase.io.read('relaxation_2.traj', ':')

# save it to an xyz
ase.io.write('full_relaxation_traj.xyz', full_traj)